# NSAID prescriptions
- Long-term treatment
- Acute treatment in previous n weeks

In [1]:
import os
import pyodbc
import pandas as pd
from IPython.display import display, Markdown
from ebmdatalab import bq

In [2]:
server = 'covid.ebmdatalab.net,1433'
database = 'OPENCoronaExport' 
username = 'SA'
password = 'my_secret_password' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### Get BNF codes mapped to snomed codes
- taken from https://github.com/ebmdatalab/nsaid-covid-codelist-notebook/blob/2432fc3bd714b4ba5aaaf8b724dff031faa52644/notebooks/nsaid.codelist.ipynb

In [3]:
sql = '''WITH bnf_codes AS (
  SELECT bnf_code FROM hscic.presentation WHERE 
  bnf_code LIKE '1001010%' #bnf section non-steroidal anti-inflammatory drugs
  )

SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, bnf_code, id'''

nsaid_codelist = bq.cached_read(sql, csv_path=os.path.join('..','data','nsaid_codelist.csv'))
nsaid_codelist.head(10)

,type,id,bnf_code,nm
0,amp,92511000001106,100101040AAAAAA,Tenoxicam 20mg tablets (A A H Pharmaceuticals ...
1,amp,565511000001103,100101040AAAAAA,Tenoxicam 20mg tablets (Sandoz Ltd)
2,amp,602711000001108,100101040AAAAAA,Tenoxicam 20mg tablets (Alliance Healthcare (D...
3,amp,753111000001104,100101040AAAAAA,Tenoxicam 20mg tablets (Kent Pharmaceuticals Ltd)
4,amp,15229011000001105,100101040AAAAAA,Tenoxicam 20mg tablets (Sigma Pharmaceuticals ...
5,amp,21786211000001109,100101040AAAAAA,Tenoxicam 20mg tablets (Waymade Healthcare Plc)
6,amp,14946111000001109,100101040AAADAD,Tenoxicam 20mg powder and solvent for solution...
7,amp,15023011000001101,100101040AAADAD,Tenoxicam 20mg powder and solvent for solution...
8,amp,15151911000001109,100101040AAADAD,Tenoxicam 20mg powder and solvent for solution...
9,amp,213111000001102,100101040BBAAAA,Mobiflex 20mg tablets (Mylan)


In [4]:
def codes_to_sql_where(col_name, code_list):
    where = ""
    i = 0
    for code in code_list:
        if i == 0:
            where = where + f"{col_name} = '{code}'"
        else:
            where = where + f" OR {col_name} = '{code}'"
        i+=1
    return where

In [5]:
codes_to_sql_where("DMD_ID", nsaid_codelist['id'].tail())

"DMD_ID = '329896000' OR DMD_ID = '35918311000001105' OR DMD_ID = '329910006' OR DMD_ID = '400480005' OR DMD_ID = '32778411000001103'"

### Count NSAID prescribing events within specified time periods

In [6]:
codes_where = codes_to_sql_where("DMD_ID", nsaid_codelist['id'])
query = f'''
SELECT 
  med.Patient_ID,
  SUM(CASE WHEN (StartDate BETWEEN '2019-04-01' AND '2020-03-31') THEN 1 ELSE 0 END) AS nsaid_long_term,
  SUM(CASE WHEN (StartDate BETWEEN '2020-03-01' AND '2020-03-31') THEN 1 ELSE 0 END) AS nsaid_acute

FROM MedicationDictionary AS dict

INNER JOIN MedicationIssue AS med
ON dict.MultilexDrug_ID = med.MultilexDrug_ID

WHERE ({codes_where})

GROUP BY med.Patient_ID

ORDER BY Patient_ID
'''
df = pd.read_sql(query, cnxn, index_col='Patient_ID')
df.loc[df['nsaid_long_term']>0]

,nsaid_long_term,nsaid_acute
Patient_ID,,
1511,3,0
43028,2,2
587627,1,0
1090214,1,0
1090876,3,0
1225111,1,0
1316333,1,0
1324281,1,0
1342856,1,0


In [7]:
cnxn.close()